## Imports

In [ ]:
import mickey.mickey
import mickey.plot
import tqdm
import numpy as np
import scipy.optimize
%pylab inline

## Directory

In [ ]:
# In order from the densest to the least dense [0,1,2,3,4,5]

my_dir = ["/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho1.5e-6/",
         "/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho1e-6/",
         "/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho5e-7/",
         "/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho3e-7/",
         "/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho1e-7/"]

In [ ]:
# Choose the directory in the next variable
directory = 0
%cd {my_dir[directory]}

## Averaging the density using regrid

In [ ]:
snap_i = 2423 #2634
snap_f = 2778 #2989
snap_range = (snap_f - snap_i)

c=mickey.mickey.Pluto(snap_i,stdout=False).regridFast(n=1500, xlim=100)

#variables
rho_avg = log10(c.rho)

In [ ]:
for i in tqdm.tqdm(range(snap_i+1,snap_f)):
    # reads data
    c=mickey.mickey.Pluto(i,stdout=False).regridFast(n=1500, xlim=100)
    
    # sum of arrays to be averaged in log10
    rho_avg = np.add(rho_avg, log10(c.rho))

rho_avg /= snap_range

## Eliminating the cold, thin, dense disk

In [ ]:
rho_avg_corona = rho_avg.copy()
rho_avg_corona[rho_avg_corona > -0.5] = -0.5

## Plotting the average variable

In [ ]:
# Check if it is everything ok with variable
r_range = 100.

figure(figsize=(8,4))

subplot(1,2,1)
pcolormesh(c.X1, c.X2, rho_avg, vmin=amin(rho_avg), vmax=amax(rho_avg), cmap='hot_r')
title("$<\\rho>$ before the threshold")
circle2=Circle((0,0),1,color='k')
gca().add_artist(circle2)
xlim(0,r_range)
ylim(-r_range/2.,r_range/2.)
colorbar()
tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)

subplot(1,2,2)
pcolormesh(c.X1, c.X2, rho_avg_corona, vmin=amin(rho_avg), vmax=amax(rho_avg), cmap='hot_r')
title("$<\\rho>$ after the threshold")
circle2=Circle((0,0),1,color='k')
gca().add_artist(circle2)
xlim(0,r_range)
ylim(-r_range/2.,r_range/2.)
colorbar()

tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)
tight_layout()

#lt.show()
plt.savefig('/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/threshold-applied.png', dpi=300)
plt.clf()
plt.cla()
plt.close('all')

## Find needed index

In [ ]:
def find_idx(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
# z-axis
idx_i = find_idx(c.x2, -25.)
idx_f = find_idx(c.x2, 25.)
# x-axis
idx_inner_boundary = find_idx(c.x1, 2.)

## Fitting Gaussian to find H (i.e. the height of the corona)

In [ ]:
exp_rho_avg_corona = 10**rho_avg_corona

In [ ]:
#Gaussian
def gaussian(x, amp, mean, sigma):
    return amp*(1/(sigma*(np.sqrt(2*np.pi))))*(np.exp(-((x-mean)**2)/(2*(sigma)**2)))

#Holds aspect ratio
H_corona, H_corona_error = [], []

for i in tqdm.tqdm(range(len(c.x1[idx_inner_boundary::]))):

    z = c.x2[idx_i:idx_f]
    rho_i = exp_rho_avg_corona.T[i][idx_i:idx_f]
    n = len(z)
    mean = sum(z * rho_i)/n
    sigma = sum(rho_i*(z-mean)**2)/n
    amp = rho_i.max()

    fit, cov = scipy.optimize.curve_fit(gaussian, z, rho_i, p0=[amp, mean, sigma], maxfev=100000)
    H_corona.append(fit[2])
    H_corona_error.append(np.sqrt(np.diag(cov))[2])

In [ ]:
print(2*max(H_corona))
idx_max_H = find_idx(H_corona, max(H_corona))
print(c.x1[idx_max_H], 2*H_corona_error[idx_max_H])

In [ ]:
H_corona = np.array(H_corona)

In [ ]:
fit_curve = gaussian(c.x2[idx_i:idx_f], fit[0], fit[1], fit[2])

plot(c.x2[idx_i:idx_f], rho_i, label="Data")
plot(c.x2[idx_i:idx_f], fit_curve, label="Gaussian fit")
plot(np.ones(2)*(fit[1]-2*fit[2]),[np.amin(fit_curve), np.amax(fit_curve)], ":k")
plot(np.ones(2)*(fit[1]+2*fit[2]),[np.amin(fit_curve), np.amax(fit_curve)], ":k")
#fill_betweenx([np.amin(fit), np.amax(fit)], popt[1]-popt[2], popt[1]+popt[2], alpha="0.1", color="black")

#plt.xlim(-2,3)

#title("Gaussian fit", fontsize=16)

#xscale('log')
#yscale('log')

xlabel("$Z/R_s$", fontsize=15)
ylabel(r"$\rho$ (code units)", fontsize=15)

plt.legend(fontsize=13)

tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)
tight_layout()

#plt.savefig('/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/fitting_gaussian.png', dpi=300)
#plt.clf()
#plt.close('all')
plt.show()

In [ ]:
plot(c.x1[idx_inner_boundary::], 2*H_corona)

xlabel("$R/R_s$", fontsize=15)
ylabel("$H/R_s$", fontsize=15)

#plt.legend(fontsize=13)

tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)
tight_layout()


#plt.savefig('/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/fitting_gaussian.png', dpi=300)
#plt.clf()
#plt.close('all')
plt.show()

## Saving the values of H for each simulation

In [ ]:
height_corona = [8.1, 9.0, 11.7, 13.0] #21.5
height_corona2 = [6.7, 7.5, 9.5, 11.2] #18.5
height_corona_error = [0.1, 0.1, 0.1, 0.2] #0.3
radius_at_height = [20.2, 21.2, 28.1, 33.8] #53.6
radius_at_height2 = [41.2, 42.4, 56.2, 66.4] #58.9
radius_at_height_err = np.zeros(4)
mdot = [0.35, 0.24, 0.11, 0.06]
mdot_err = [0.09, 0.07, 0.02, 0.01]

In [ ]:
figure(figsize=(8,4))
subplot(1,2,1)
plt.errorbar(x=mdot, y=height_corona2, yerr=height_corona_error, xerr=mdot_err,
             fmt='bo',
             capsize=3)
title("Height", fontsize=16)
xlabel("$\dot{M}/\dot{M}_{\\rm Edd}$", fontsize=15)
ylabel("$Z/R_s$", fontsize=15)
xscale("log")
#yscale("log")
#ylim(6, 11)
tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)

subplot(1,2,2)
plt.errorbar(x=mdot, y=radius_at_height2, yerr=radius_at_height_err, xerr=mdot_err, fmt='bo', capsize=3)
title("Extension", fontsize=16)
xlabel("$\dot{M}/\dot{M}_{\\rm Edd}$", fontsize=15)
ylabel("$R/R_s$", fontsize=15)
xscale("log")
#yscale("log")
xlim(0.04)
#ylim(20, 60)
tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)
tight_layout()

#plt.legend(fontsize=13)
#plt.show()
plt.savefig('/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/height_extension_corona.png', dpi=300)
plt.clf()
plt.close('all')

## Temperature of the corona

In [ ]:
temperature_convertion = 3.4859007e12 #kelvin * mu

In [ ]:
snap_i = 2423 #2634
snap_f = 2778 #2989
snap_range = (snap_f - snap_i)

p=mickey.mickey.Pluto(snap_i,stdout=False)

#variables
temperature_avg = log10(p.p/p.rho*temperature_convertion)
tracer_avg = p.tr1

for i in tqdm.tqdm(range(snap_i+1,snap_f)):
    # reads data
    p=mickey.mickey.Pluto(i,stdout=False)
    
    # sum of arrays to be averaged in log10
    temperature_avg = np.add(temperature_avg, log10(p.p/p.rho*temperature_convertion))
    tracer_avg = np.add(tracer_avg, p.tr1)

temperature_avg /= snap_range
tracer_avg /= snap_range

In [ ]:
tracer_avg_copy = tracer_avg.copy()
tracer_avg_copy[tracer_avg_copy < 0.9] = 0

In [ ]:
# Check if it is everything ok with variable
r_range = 100.

figure(figsize=(8,6))

#subplot(1,2,1)
mappable2=pcolormesh(p.X, p.Y, tracer_avg.T, vmin=0, vmax=1., cmap='binary')
mappable1=pcolormesh(p.X, p.Y, temperature_avg.T, vmin=7.5, vmax=12., cmap='hot_r')
c1=colorbar(mappable1)
c1.set_label(label="log10($T$/K)", fontsize=16)
plt.contour(p.X, p.Y, tracer_avg.T, cmap='binary')
c2=colorbar(mappable2)
c2.set_label(label="Tracer", fontsize=16)
#title("$<T>$", fontsize=16)
circle2=Circle((0,0),1,color='k')
gca().add_artist(circle2)
xlim(0,r_range)
ylim(-r_range/2.,r_range/2.)
tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)

'''
subplot(1,2,2)
pcolormesh(p.X, p.Y, tracer_avg_copy.T, vmin=0., vmax=1., cmap='hot')
title("<tracer>")
circle2=Circle((0,0),1,color='k')
gca().add_artist(circle2)
xlim(0,r_range)
ylim(-r_range/2.,r_range/2.)
colorbar()

tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)
'''
tight_layout()


plt.show()
#plt.savefig('/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/temperature-tracer-threshold.png', dpi=300)
plt.clf()
plt.cla()
plt.close('all')

In [ ]:
temperature_avg_copy = temperature_avg.copy()

for i in range(len(temperature_avg)):
    for j in range(len(temperature_avg)):
        if tracer_avg[i,j] < 0.9 or p.x1[i] > radius_at_height2[directory] or temperature_avg[i,j] < 9.:
            temperature_avg[i,j] = 0.

In [ ]:
# Recovering original values
temperature_avg = temperature_avg_copy.copy()

In [ ]:
temperature_avg = temperature_avg[temperature_avg > 9.]
temperature_avg = 10**temperature_avg

In [ ]:
print("%.2e %.2e %0.2e" % (mean(temperature_avg), median(temperature_avg), std(temperature_avg)))
print("%.2e %.2e %0.2e" % (10**(mean(log10(temperature_avg))),
                           10**(median(log10(temperature_avg))), 
                           10**(std(log10(temperature_avg)))))

In [ ]:
mean_corona_temperature = [4.92e10, 5.06e10, 5.76e10, 6.59e10] #threshold < 9.; linear mean
median_corona_temperature = [3.30e10, 3.39e10, 3.81e10, 4.48e10]
std_corona_temperature = [4.47e10, 4.63e10, 5.53e10, 6.36e10]

In [ ]:
figure(figsize=(8,8))

plt.errorbar(x=mdot, y=median_corona_temperature, yerr=std_corona_temperature, xerr=mdot_err,
             fmt='bo',
             capsize=3)
#title("<$T$/K>", fontsize=16)
xlabel("$\dot{M}/\dot{M}_{\\rm Edd}$", fontsize=15)
ylabel("$\log10(T/\mathrm{K})$", fontsize=15)
xscale("log")
yscale("log")
#ylim(10.5, 10.67)
tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)

tight_layout()

plt.show()
#plt.savefig('/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/mean-T-corona.png', dpi=300)
plt.clf()
plt.close('all')